In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import sys

def main():
    # 1. Configuration des arguments de la ligne de commande
    parser = argparse.ArgumentParser(description="Analyse de données pays par groupes.")
    
    # Arguments obligatoires
    parser.add_argument('fichier_csv', type=str, help="Le chemin vers le fichier CSV")
    parser.add_argument('col_groupe', type=str, help="Argument 1 : Colonne pour grouper (ex: Niveau de revenus)")
    parser.add_argument('col_valeur', type=str, help="Argument 2 : Colonne à moyenner (ex: Espérance de vie)")
    
    # Argument optionnel pour le label de l'ordonnée
    parser.add_argument('--ylabel', type=str, default='Valeur Moyenne', help="Texte à afficher sur l'axe des ordonnées (Y)")

    args = parser.parse_args()

    # 2. Chargement des données
    try:
        # On essaie de lire le CSV. Si ton fichier utilise des points-virgules, change sep=',' en sep=';'
        df = pd.read_csv(args.fichier_csv, sep=',') 
        print(f"--> Fichier '{args.fichier_csv}' chargé avec succès ({len(df)} lignes).")
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{args.fichier_csv}' est introuvable.")
        sys.exit(1)

    # 3. Vérification des colonnes
    if args.col_groupe not in df.columns or args.col_valeur not in df.columns:
        print(f"Erreur : Une des colonnes spécifiées n'existe pas dans le CSV.")
        print(f"Colonnes disponibles : {list(df.columns)}")
        sys.exit(1)

    # 4. Nettoyage et Calculs
    # On retire les lignes où la colonne de groupe est vide (NaN)
    df_clean = df.dropna(subset=[args.col_groupe])
    
    # On s'assure que la colonne valeur est bien numérique
    df_clean[args.col_valeur] = pd.to_numeric(df_clean[args.col_valeur], errors='coerce')
    
    # Groupement et calcul de la moyenne
    # On fait la moyenne, puis on reset_index pour faciliter le plot
    resultat = df_clean.groupby(args.col_groupe)[args.col_valeur].mean().sort_values()

    print("\n--> Résultats calculés (Moyennes par groupe) :")
    print(resultat)

    # 5. Création du graphique (Bar Chart)
    plt.figure(figsize=(10, 6)) # Taille de la figure
    
    # Création des bâtons
    # resultat.index contient les noms des groupes, resultat.values les moyennes
    plt.bar(resultat.index, resultat.values, color='skyblue', edgecolor='black')

    # Mise en forme
    plt.xlabel(args.col_groupe, fontsize=12, fontweight='bold') # Abscisse
    plt.ylabel(args.ylabel, fontsize=12, fontweight='bold')     # Ordonnée (personnalisée)
    plt.title(f"Moyenne de '{args.col_valeur}' par '{args.col_groupe}'", fontsize=14)
    
    # Rotation des étiquettes en bas si elles sont longues
    plt.xticks(rotation=45, ha='right')
    
    # Grille en arrière-plan pour la lisibilité
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Ajustement automatique des marges
    plt.tight_layout()

    # Affichage
    print("--> Affichage du graphique...")
    plt.show()

if __name__ == "__main__":
    main()